# **IMPORTS**

In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import  drive
drive.mount('/drive')

!pip install faker
from faker import Faker
faker = Faker()
import random


Mounted at /drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.8 MB/s 


# **DATA VALUES**

In [ ]:
# VARIABLES TO BE FILLED
property_price = None
property_type = None
property_area_sqft = None
property_area_sqm = None
property_bedrooms = None
property_bathrooms = None
property_id = None
property_listing_date = None
property_location = None
property_desc = None
property_amenities = None

agent_name = None
agent_phone = None
agent_email = None
agent_whatsapp = None

broker_name = None
broker_listings = None
broker_address = None
broker_phone = None

project_name = None
project_location = None
project_head_office = None
project_total_units = None
project_photo = None

# **LOOPING THROUGH ALL PAGES**

In [ ]:
property_data = []
agent_data = []
broker_data = []
project_data = []
user_data = []
review_data = []

# SET CONDITIONS FOR LOOP
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/106.0.1370.42'}

max_listing_date = 14
pageno = 1
listing_date_below_14 = True

# #####                                  #######
# #####  LOOP THROUGH ALL PAGES          ######
# #####                                  #######



while listing_date_below_14:
  # GET PAGE URL
  page_url = "https://www.propertyfinder.eg/en/search?c=1&ob=nd&page="+str(pageno)+"&pt=3000000"
  
  # GET PAGE RESPONSE
  page_response = requests.get(page_url, headers=HEADERS)
  
  # GET PAGE SOUP
  page_soup = BeautifulSoup(page_response.content)
  
  # GET FIRST LISTING DATE

  first_listing_date = page_soup.find(class_ = "card-footer__publish-date")
  print("PAGE NUMBER: " + str(pageno))

  print(first_listing_date.text)

  # CHECK LISTING DATE IS NOT GREATER THAN 14

  if "14 days ago" in first_listing_date.text:
    listing_date_below_14 = False
    break
  
  #  FOR LOOP THROUGH ALL LISTINGS ON THE PAGE

  for i in page_soup.find_all(class_ = "card__link"):
    
    # GET LISTING URL, RESPONSE, SOUP
    listing_url = "https://www.propertyfinder.eg" + i.get('href')
    print("#############LISTING ON PAGE " + str(pageno) + "####################")
    print("LISTING URL: "+listing_url)

    listing_response = requests.get(listing_url, headers=HEADERS)
    listing_soup = BeautifulSoup(listing_response.content)

    # #####                                  #######
    # #####  GET DATA FOR PROPERTY           ######
    # #####                                  #######


    # PROPERTY PRICE
    try:
      property_price = listing_soup.find(class_ = "property-price__price").text.replace("\t", "").replace("\n", "").replace(" ", "").replace(",", "").replace("EGP", "")
      print("PROPERTY PRICE: " + property_price)
    except:
      property_price = None
      print("COULD NOT OBTAIN VALUE ######### PROPERTY PRICE")


    # PROPERTY TYPE
    try:
      property_facts = listing_soup.find_all(class_ = "property-facts__value")
      property_type = property_facts[0].text.replace("\t", "").replace("\n", "").replace("\xa0/\xa0", " / ").replace("+Maid", "")
      print("Type: "+ property_type)
    except:
      property_facts = None
      print("COULD NOT OBTAIN VALUE ######### PROPERTY FACTS")

    # PROPERTY AREA

    try:
      property_area_sqft = property_facts[1].text.replace("\t", "").replace("\n", "").replace("\xa0/\xa0", " / ").replace(",", "").split()[0].replace('sqft', '')
      property_area_sqm =  property_facts[1].text.replace("\t", "").replace("\n", "").replace("\xa0/\xa0", " / ").replace(",", "").split()[2].replace('sqm', '')
      print("SQFT: "+ property_area_sqft)
      print("SQM: " + property_area_sqm)
    except:
      property_area_sqft = None
      property_area_sqm = None
      print("COULD NOT OBTAIN VALUE ######### PROPERTY AREA")

    # PROPERTY BEDROOMS
    try:
      property_bedrooms = property_facts[2].text.replace("\t", "").replace("\n", "").replace("\xa0/\xa0", " / ").replace("+Maid", "")
      print("Bedrooms: "+ property_bedrooms)
    except:
      property_bedrooms = None
      print("COULD NOT OBTAIN VALUE ######### BEDROOMS")

    # PROPERTY BATHROOMS
    try:
      property_bathrooms = property_facts[3].text.replace("\t", "").replace("\n", "").replace("\xa0/\xa0", " / ").replace("+Maid", "")
      print("Bathrooms: "+ property_bathrooms)
    except: 
      print("COULD NOT OBTAIN VALUE ######### BATHROOMS")
      property_bathrooms = None


    # PROPERTY ID
    try:
      property_id = listing_soup.find_all(class_ = 'property-page__legal-list-content')[0].text.replace("\\", "").replace( "\n", "").replace("\n", "")
      print("ID: "+ property_id)
    except:
      property_id = None
      print("COULD NOT OBTAIN VALUE ######### PROPERTY ID")

    # PROPERTY LISTING DATE
    try:
      property_listing_date = listing_soup.find_all(class_ = 'property-page__legal-list-content')[1].text
      print("LISTING DATE: " + property_listing_date)
    except:
      property_listing_date = None
      print("COULD NOT OBTAIN VALUE ######### LISTING DATE")


    # LOCATION
    try:
      property_location = list(listing_soup.find(class_ = "property-location__detail-area").children)[1].text
      print("LOCATION: "+ property_location)
    except:
      print("COULD NOT OBTAIN VALUE ######### LOCATION")
      property_location = None


    # DESCRIPTION
    try:
      property_desc = listing_soup.find(class_ =  "property-description__text-trim").text
      print("PROPERTY DESCRIPTION: AVAILABLE")
    except:
      print("COULD NOT OBTAIN VALUE ######### DESCRIPTION")
      property_desc = None
    


    # AMENITIES

    try:
      property_amenities = listing_soup.find(class_ = 'property-amenities').text
      print("AMENITIES: AVAILABLE")
    except:
      property_amenities = None
      print("COULD NOT OBTAIN VALUE ######### AMENITIES")
      
    # #####                                  #######
    # #####  GET DATA FOR AGENT              ######
    # #####                                  #######


    # AGENT NAME
    try:
      agent_name = listing_soup.find(class_="text text--size3 property-agent__name").text.replace("..","")
      print("AGENT NAME: "+ agent_name)
    except:
      agent_name = None
      print("COULD NOT OBTAIN VALUE ######### AGENT NAME")


    all_scripts = listing_soup.find_all('script')
    script_string=str(all_scripts[15])
    # AGENT PHONE
    try:
      agent_phone = "+20"+re.search("phone\",\"value\":\"\+20(.{10})", script_string).group(1)
      print("AGENT PHONE: "+ agent_phone)
    except:
      agent_phone = None
      print("COULD NOT OBTAIN VALUE ######### AGENT PHONE")

    # AGENT EMAIL
    try:
      agent_email =  re.search("email\",\"value\":\"(.{23})", script_string).group(1)
      print("AGENT EMAIL: "+agent_email)
    except:
      agent_email = None
      print("COULD NOT OBTAIN VALUE ######### AGENT EMAIL")

    # AGENT WHATSAPP
    try:
      agent_whatsapp = "+20" + re.search("whatsapp\",\"value\":\"\+20(.{10})", script_string).group(1)
      print("AGENT WHATSAPP: "+ agent_whatsapp)
    except:
      agent_whatsapp = None
      print("COULD NOT OBTAIN VALUE ######### AGENT WHATSAPP")
        
    # #####                                  #######
    # #####  GET DATA FOR BROKER COMPANY     ######
    # #####                                  #######

    try:
      broker_company_list = list(listing_soup.find(class_ = 'text text--size2 property-agent__position').children)
    except:
      broker_company_list = None
      print("COULD NOT OBTAIN VALUE ######### BROKER COMPANY LIST")
    # BROKER NAME
    
    try:
      broker_name = broker_company_list[0].text.strip() # broker company name
      print("BROKER NAME: " + broker_name)
    except:
      broker_name = None
      print("COULD NOT OBTAIN VALUE ######### BROKER NAME")

    # BROKER no OF LISTINGS
    broker_listings = None
    try:
      broker_listings = int(broker_company_list[1].text.strip().replace("(", "").replace(")", "").replace(",", "").replace("properties", "")) # broker company no of listings
      print("BROKER LISTINGS: "+ str(broker_listings))
    except:
      broker_listings = None
      print("COULD NOT OBTAIN VALUE ######### BROKER NO OF LISTINGS")
    

    # GET BROKER COMPANY PAGE LINK
    broker_company_link = ""
    try:
      for i in listing_soup.find(class_="property-agent__broker-image-area"):
        if i.get('href') != None:
          broker_company_link = 'https://www.propertyfinder.eg' + str(i.get('href'))
    except:
      print("COULD NOT OBTAIN VALUE ######### BROKER COMPANY PAGE LINK")


    # GET BROKER PAGE REQUEST AND SOUP

    try:
      broker_page =  requests.get(broker_company_link, headers=HEADERS)
      broker_soup = BeautifulSoup(broker_page.content)
    except:
      broker_page = None
      broker_soup = None
      print("COULD NOT OBTAIN VALUE ######### BROKER PAGE AND SOUP")

    # BROKER HEAD OFFICE ADDRESS
    
    try:
      broker_address = broker_soup.find(class_ = 'bio-info__details').text.replace("Head office:", "")
      print("BROKER ADDRESS: "+broker_address)
    except:
      print("COULD NOT OBTAIN VALUE ######### BROKER ADDRESS")
      broker_address = None


    # BROKER PHONE
    
    try:
      broker_phone = broker_soup.find(class_ = 'button__phone-ltr').text
      print("BROKER PHONE: "+broker_phone)
    except:
      print("COULD NOT OBTAIN VALUE ######### BROKER PHONE")
      broker_phone = None

    # #####                                   #######
    # #####  GET DATA FOR DEVELOPMENT PROJECT ######
    # #####                                   #######

    # DEVELOPMENT PROJECT PAGE
    
# Find project page link


   # TEST LISTING SOUP - LINK WITH A PROJECT
    # listing_url = 'https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-cairo-shorouk-city-5th-district-cleopatra-palace-991757.html'
    # listing_response = requests.get(listing_url, headers=HEADERS)
    # listing_soup = BeautifulSoup(listing_response.content)


    project_url = None
    project_response = None
    project_soup = None
    project_name = None
    project_location = None
    project_head_office = None
    project_total_units = None
    project_status = None
    project_photo = None

    if listing_soup.find(class_ = 'property-project-details__title') != None:
      property_list = list(listing_soup.find(class_="property-project-details__developer-box").children)
      property_list_2 = list(property_list[1].children)
      project_url = 'https://www.propertyfinder.eg' + str(property_list_2[1].get('href'))

      print("Project URL: "+ project_url)


      # GET SOUP OF PROJECT PAGE
      project_response = requests.get(project_url, headers=HEADERS)
      project_soup = BeautifulSoup(project_response.content)



      # GET PROJECT NAME - listing page
      try:
        project_name = listing_soup.find(class_ = 'property-project-details__title').text
      except Exception as e:
        print(e)

      # GET PROJECT LOCATION - listing page
      try:
        project_location = listing_soup.find(class_='property-project-details__location').text

      except Exception as e:
        print(e)
        print("COULD NOT OBTAIN VALUE ######### PROJECT LOCATION")


      # GET PROJECT STATUS - listing page

      # --- use listing soup
      try:
        project_status = listing_soup.find_all(class_ = 'property-project-details__list-item-value')[3].text.strip()
      except Exception as e:
        print(e)
        print("PROJECT STATUS: "+project_status)

      # GET PROJECT PHOTO - listing page
      try:
        project_photo = listing_soup.find(class_ ='property-project-details-gallery__image-container-img').get('src')
      except Exception as e:
        print(e)

      # PROJECT HEAD OFFICE - project page
      try:
        project_head_office = list(list(project_soup.find(class_ = "bio-info__details").children)[0].children)[1].text.strip()
      except Exception as e:
        print(e)
          

      # GET PROJECT TOTAL UNITS - project page
      try:
        project_total_units = project_soup.find_all("a", { "class" : "link"})[1].text.replace(" Properties","")
      except Exception as e:
        print(e)

      
      print(project_name) 
      print(project_location)
      print(project_head_office)
      print(project_total_units)
      print(project_status)
      print(project_photo)

    else:
      print("No Project Found")


    property_data.append({
    "id": property_id,
    "price": property_price,
    "area_sqft": property_area_sqft,
    "area_sqm": property_area_sqm,
    "bedrooms": property_bedrooms,
    "bathrooms": property_bathrooms,
    "date": property_listing_date,
    "location": property_location,
    "description": property_desc,
    "amenities": property_amenities,
    "agent_phone": agent_phone,
    "project_name": project_name
    })
    
    agent_data.append({
        "name": agent_name,
        "phone": agent_phone,
        "email": agent_email,
        "whatsapp": agent_whatsapp,
        "broker_phone": broker_phone
    })

    broker_data.append({
        "name": broker_name,
        "listings": broker_listings,
        "address": broker_address,
        "phone": broker_phone
    })

    project_data.append({
        "name": project_name,
        "location": project_location,
        "head_office": project_head_office,
        "total_units": project_total_units,
        "photo": project_photo,
        "status": project_status
    })
    
    username = faker.name()
    review_id = faker.ean()

    user_data.append({
        "username": username,
        "email": faker.email(),
        "gender": random.choices(["M","F"]),
        "age": random.randrange(18,65),
        "birthdate": str(faker.date_of_birth()),
        "review_id": review_id,
        "area_of_focus": str(faker.word())
    })
    
    review_data.append({
        "review_id": review_id,
        "username": username,
        "agent_phone_no": agent_phone,
        "date": str(faker.date_this_year())
    })


  property_table=pd.DataFrame(property_data)
  agent_table = pd.DataFrame(agent_data)
  broker_table = pd.DataFrame(broker_data)
  project_table = pd.DataFrame(project_data)
  user_table = pd.DataFrame(user_data)
  review_table = pd.DataFrame(review_data)

  print(property_table)
  print(agent_table)
  print(broker_table)
  print(project_table)
  print(user_table)
  print(review_table)
  

  
  pageno+=1



# **ASSUMPTIONS / CONSTRAINTS**

In [ ]:

# ASSUMPTIONS / CONSTRAINTS

"""

* The reference of each listing effectively is unique and acts as an identifying primary key
* No two projects have the same name, so name can be used as a project identifier
* Amenities are contained in a single string, so it does not require a table to store it
* 

"""
